# AWS BEDROCK API EXAMPLE

In [25]:
! pip install boto3

In [26]:
import boto3
from botocore.exceptions import ClientError
import os
import json
import logging

from dotenv import load_dotenv, find_dotenv

logger = logging.getLogger(__name__)

load_dotenv(find_dotenv())
aws_access_key_id = os.environ.get("aws_access_key_id")
aws_secret_access_key = os.environ.get("aws_secret_access_key")

### Generative AI with large language models available on AWS

In [27]:
def generate(prompt: str) -> str:
    boto3_bedrock = boto3.client(
        'bedrock-runtime',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="eu-central-1"
    )

    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 300})

    modelId = 'anthropic.claude-instant-v1'
    contentType = 'application/json'

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    return response_body['completion']

response = generate("Human:When second world war ended? Assistant:")
print(response)

 The Second World War ended in 1945. Here are some key dates regarding the end of WWII:

- May 8, 1945 - Germany surrendered unconditionally, signing the German Instrument of Surrender at Allied headquarters in Reims, France, ending the war in Europe (VE Day). 

- August 14, 1945 - Japan surrendered unconditionally following the dropping of atomic bombs on Hiroshima and Nagasaki by the United States, ending the war in the Pacific and World War II overall (VJ Day).

- September 2, 1945 - Formal surrender ceremonies were held on the deck of the USS Missouri in Tokyo Bay, Japan. This is known as V-J Day and marked the official end of World War II.

So in summary, the war in Europe ended in early May 1945 with Germany's surrender, the war in the Pacific ended in mid-August 1945 with Japan's surrender after the atomic bombings, and the formal end to all hostilities of WWII took place with Japan's surrender ceremony on September 2, 1945 aboard the USS Missouri. The entire global conflict spa

#### Multimodal LLM

In [31]:
import base64

def generate_multimodal(image_path: str, prompt: str):
    boto3_bedrock = boto3.client(
        'bedrock-runtime',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="eu-central-1")
    image_base64 = base64.b64encode(open(image_path, 'rb').read()).decode('utf-8')
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": image_base64
                            }
                        },
                        {
                            "type": "text",
                            "text": prompt
                        }
                    ]
                }
            ]
        }
    )

    modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
    contentType = 'application/json'

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    return response_body

response = generate_multimodal('img/satellite.png', "count cars on the image. Return only number without any additional text.")

print(response)

{'id': 'msg_bdrk_01Ge2JCTeNiNEqMgJspXHo3W', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-5-sonnet-20240620', 'content': [{'type': 'text', 'text': '30'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1482, 'output_tokens': 5}}


### Get model details via API

In [20]:
from lib.bedrock import BedrockWrapper


def usage_demo():
    """
    Shows how to list the available foundation models.
    This demonstration gets the list of available foundation models and
    prints their respective summaries.
    """
    logging.basicConfig(level=logging.INFO)
    print("-" * 88)
    print("Welcome to the Amazon Bedrock demo.")
    print("-" * 88)

    bedrock_client = boto3.client(
        service_name="bedrock",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="eu-central-1"
    )

    wrapper = BedrockWrapper(bedrock_client)

    print("Listing the available foundation models.")

    try:
        for model in wrapper.list_foundation_models():
            print_model_details(model)
    except ClientError:
        logger.exception("Couldn't list foundation models.")
        raise

    print("Getting the details of an individual foundation model.")

    model_id = "amazon.titan-embed-text-v1"

    try:
        print_model_details(wrapper.get_foundation_model(model_id))
    except ClientError:
        logger.exception(f"Couldn't get foundation model {model_id}.")
        raise


def print_model_details(model):
    print("\n" + "=" * 42)
    print(f' Model: {model["modelId"]}')
    print("-" * 42)
    print(f' Name: {model["modelName"]}')
    print(f' Provider: {model["providerName"]}')
    print(f' Model ARN: {model["modelArn"]}')
    print(f' Lifecycle status: {model["modelLifecycle"]["status"]}')
    print(f' Input modalities: {model["inputModalities"]}')
    print(f' Output modalities: {model["outputModalities"]}')
    print(f' Supported customizations: {model["customizationsSupported"]}')
    print(f' Supported inference types: {model["inferenceTypesSupported"]}')
    if "responseStreamingSupported" in model:
        print(f' Response streaming supported: {model["responseStreamingSupported"]}')

    print("=" * 42)

usage_demo()

----------------------------------------------------------------------------------------
Welcome to the Amazon Bedrock demo.
----------------------------------------------------------------------------------------
Listing the available foundation models.


INFO:bedrock:Got 27 foundation models.



 Model: amazon.titan-text-express-v1:0:8k
------------------------------------------
 Name: Titan Text G1 - Express
 Provider: Amazon
 Model ARN: arn:aws:bedrock:eu-central-1::foundation-model/amazon.titan-text-express-v1:0:8k
 Lifecycle status: ACTIVE
 Input modalities: ['TEXT']
 Output modalities: ['TEXT']
 Supported customizations: []
 Supported inference types: []
 Response streaming supported: True

 Model: amazon.titan-text-express-v1
------------------------------------------
 Name: Titan Text G1 - Express
 Provider: Amazon
 Model ARN: arn:aws:bedrock:eu-central-1::foundation-model/amazon.titan-text-express-v1
 Lifecycle status: ACTIVE
 Input modalities: ['TEXT']
 Output modalities: ['TEXT']
 Supported customizations: []
 Supported inference types: ['ON_DEMAND']
 Response streaming supported: True

 Model: amazon.titan-text-lite-v1:0:4k
------------------------------------------
 Name: Titan Text G1 - Lite
 Provider: Amazon
 Model ARN: arn:aws:bedrock:eu-central-1::foundation-m